### Cart (Classification and Regression Trees) and Random Forest

•	Recursively partitioning the input space, and defining a local model in each resulting region of input space.

•	To make a prediction for a given observation, the method typically use the mean or the mode of the training observations in the region to which it belongs.


Alternative approach is to sub-divide or **partition** the space into smaller regions, and then partition the subdivision again **(recursive partitioning** process) until we get to chunks of the space that are easier to control so that we can ultimately fit simple models to them.

### Tree components

### Terms

Nodes: points on which decisions will be made

Edges: Pathways

### Structure

root node: Start

internal node: Tests conditions 

leaf node: Final output, means of the observations 

Each of the terminal nodes, or leaves, of the tree represents a cell of the partition, and has attached to it a simple model which applies in that cell only.

* Then, the model for the leaf is the the sample mean of the dependent variable in that cell:
$$\hat y = \frac1 c \sum_{i=1}^c y_i $$ 

Once the local models are completely determined, we need to find a good tree (this means finding a good partitioning of the data)

We need to do a greedy search => find the binary question that maximizes the information we get about  Y , this will create the root node and 2 daughters nodes. Then at each daughter node we repeat this procedure asking which question would give us the maximum information about  Y

### Stopping criterion

* This means stop growing the tree when further splits gives less than some minimal amount of extra information.

The sum of squared errors for a tree $T$ is:
$$S = \sum_{c \in leaves (T)}  \sum_{i \in C} (y_i - m_c)^2  $$ 
* where the prediction for leave $c$ is: 
$$m_c = \frac 1 n_c \sum_{c \in leaves C} y_i $$ 
* therefore we can rewrite $S$ as: 
$$S = \sum_{c \in leaves (T)} n_c V_C  $$ 
* where $V_c$ is the within leave variance of leaf $c$, and now we can make our splits to minimize $S$

### Tree pruning

* In the minimization process we are more likely to create good predictions on the training set, but there is a high risk of overfitting the data => leads to a poor test performance.
* To avoid this we need select a subtree that leads to a lower error rate

### Pros and Cons

| **Pros**   |      **Cons**     | 
|:----------|:-------------:|
| Simple to understand/interpret | Overfitting | 
| Little data preparation (normalization, create dummies) | Greedy algorithm |
|Easily handle mixed discrete and continuous inputs|Unstable: small changes to the input data=>large effects on the structure of the tree|
|Insensitive to monotone transformations of the input|Trees are high **variance** estimators|
|Perform automatic variable selection|
|Relatively robust to outliers|
|Robust - performs well when assumptions are violated|  
|Performs well in large datasets |    
|Extremely fast execution | 

### Random Forest

* One way to reduce variance of an estimate is to average together many estimates.
* We can train $M$ different trees on different subsets of the data, **choosing randomly with replacement** and then compute the **ensemble:**
$$ f(X)= \sum_{m=1}^M \frac 1M f_m(X)$$

where $f_m$ is the $m$’th tree. 
* This technique is called **bagging** => “bootstrap aggregating”

* Re-running the same learning algorithm on different subsets of the data can result in highly correlated predictors.
* Random forests tries to decorrelate the base learners by learning trees based on a **randomly** chosen subset of input variables, as well as a **randomly** chosen subset of data cases.

# Steps

### Step 0

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

from IPython.display import display
from sklearn.cross_validation import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Step 1

* Defining test and train samples

In [ ]:
## Define y
y = df['y']

## Define X (exclude inc, incsq, log_inc)
columns_ = df.columns.tolist()
exclude_cols = ['x1', 'x2', 'x3']

X = df[[i for i in columns_ if i not in exclude_cols]]

## Print shapes of y and X
print y.shape, X.shape

In [ ]:
## Train test split 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Print shapes of X(s) and y(s)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

### Step 1.1

Build decision tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()

## Here is the gridsearch
params = {"max_depth": [3,5,10,20],
          "max_features": [None, "auto"],
          "min_samples_leaf": [1, 3, 5, 7, 10],
          "min_samples_split": [2, 5, 7],
           "criterion" : ['mse']
         }

# ## Here crossvalidate 
from sklearn.grid_search import GridSearchCV
dtr_gs = GridSearchCV(dtr, params, n_jobs=-1, cv=5, verbose=1)

### A. Fit your tree

In [ ]:
dtr_gs.fit(X_train, y_train)

### B. Print best stimator, best parameters, and best score

In [ ]:
''' dtr_best = is the regression tree regressor with best parameters/estimators'''
dtr_best = dtr_gs.best_estimator_ 

print "best estimator", dtr_best
print "\n==========\n"
print "best parameters",  dtr_gs.best_params_
print "\n==========\n"
print "best score", dtr_gs.best_score_

### C. Print feature importances 

In [ ]:
''' Here I am defining a function to print feature importance using best models'''
def feature_importance(X, best_model):
    feature_importance = pd.DataFrame({'feature':X.columns, 'importance':best_model.feature_importances_})
    feature_importance.sort_values('importance', ascending=False, inplace=True)
    return feature_importance 

In [ ]:
feature_importance(X, dtr_best)

### D. Predict

(in the test data)

In [ ]:
y_pred_dtr= dtr_best.predict(X_test)
y_pred_dtr

### E. Evaluate the performance of you model

*MSE in train and test data, R2 in train and test data

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
''' Function that calls the MSE and R^2 at once, using the name of the method and calling the best model'''

def rsquare_meansquare_error(train_y, test_y, train_X, test_X, test, best_model):
    """ first we need to predict on the test and train data"""
    y_train_pred = best_model.predict(train_X)
    y_test_pred = best_model.predict(test_X)
    
    """ We call the MSE in the following lines"""
    print ('MSE ' + test + ' train data: %.2f, test data: %.2f' % (
        mean_squared_error(train_y, y_train_pred),
        mean_squared_error(test_y, y_test_pred)))
    
    """ We call the R^2 in the following lines"""
    print('R^2 ' + test + ' train data: %.2f, test data: %.2f' % (
        r2_score(train_y, y_train_pred),
        r2_score(test_y, y_test_pred)))

In [ ]:
rsquare_meansquare_error(y_train, y_test, X_train, X_test, "Regression tree", dtr_best)

### F. Visualize your tree using the best parameters/estimators

In [ ]:
# REQUIREMENTS:
# pip install pydotplus
# brew install graphviz

# Use graphviz to make a chart of the regression tree decision points:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
#### import pydotplus
import pydot

In [ ]:
dot_data = StringIO()
''' dtr_best was defined before in section B'''

## Graph
export_graphviz(dtr_best, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names=X.columns)  

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png()) 

### 2. Random Forest Regression Tree

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor( )

params = {'max_depth':[3,4,5], 
          'max_features':[2,3,4], 
          'max_leaf_nodes':[5,6,7], 
          'min_samples_split':[3,4],
         'n_estimators': [100]
         }

estimator_rfr = GridSearchCV(forest, params, n_jobs=-1,  cv=5,verbose=1)

### A. Fit random tree

In [ ]:
estimator_rfr.fit(X_train, y_train)

### B. Print best estimator

In [ ]:
''' rfr_best = is the random forest regression tree regressor with best parameters/estimators'''
rfr_best = estimator_rfr.best_estimator_
print "best estimator", rfr_best
print "\n==========\n"
print "best parameters", estimator_rfr.best_params_
print "\n==========\n"
print "best score", estimator_rfr.best_score_

### C. Print feature importances

In [ ]:
feature_importance(X, rfr_best)

### D. Predict

In [ ]:
y_pred_rfdtr= rfr_best.predict(X_test)
y_pred_rfdtr

### E. Evaluate the performance of your model 
* MSE in train and test data, R2 in train and test data

In [ ]:
quare_meansquare_error(y_train, y_test, X_train, X_test, "Random Forest Regression tree", rfr_best)